In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
import warnings
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split 

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
### configuration of output of cells
# pd.set_option('display.max_seq_items', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:
charging_set = pd.read_csv("data/charging_sessions.csv")

In [4]:
# general information about dataset, only use if needed for better overview of notebook
# TODO: implement it for the assignment!!!!!
# charging_set.info()
# charging_set.describe()
# charging_set.columns

charging_set.head(10)

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."
5,5,5e23b149f9af8b5fe4b973d4,2020-01-02 14:00:13+00:00,2020-01-02 23:00:41+00:00,2020-01-02 20:13:21+00:00,13.370,1_1_194_821_2020-01-02 14:00:05.241573,1,AG-1F13,1-1-194-821,America/Los_Angeles,3519.0,"[{'WhPerMile': 600, 'kWhRequested': 24.0, 'mil..."
6,6,5e23b149f9af8b5fe4b973d5,2020-01-02 14:09:14+00:00,2020-01-03 01:57:58+00:00,2020-01-03 00:59:21+00:00,43.477,1_1_178_817_2020-01-02 14:09:14.451827,1,AG-1F09,1-1-178-817,America/Los_Angeles,933.0,"[{'WhPerMile': 385, 'kWhRequested': 65.45, 'mi..."
7,7,5e23b149f9af8b5fe4b973d6,2020-01-02 14:17:32+00:00,2020-01-03 00:44:54+00:00,2020-01-02 20:22:40+00:00,32.441,1_1_191_804_2020-01-02 14:17:32.329535,1,AG-4F34,1-1-191-804,America/Los_Angeles,3737.0,"[{'WhPerMile': 300, 'kWhRequested': 45.0, 'mil..."
8,8,5e23b149f9af8b5fe4b973d7,2020-01-02 14:25:38+00:00,2020-01-02 19:36:05+00:00,2020-01-02 18:38:39+00:00,13.264,1_1_194_826_2020-01-02 14:25:37.578692,1,AG-1F11,1-1-194-826,America/Los_Angeles,419.0,"[{'WhPerMile': 400, 'kWhRequested': 40.0, 'mil..."
9,9,5e23b149f9af8b5fe4b973d8,2020-01-02 14:27:40+00:00,2020-01-02 20:01:48+00:00,2020-01-02 19:17:46+00:00,13.285,1_1_178_823_2020-01-02 14:27:39.582337,1,AG-1F08,1-1-178-823,America/Los_Angeles,651.0,"[{'WhPerMile': 400, 'kWhRequested': 20.0, 'mil..."


# TODO

- check validity of columns
- check all columns for wrong data types
- check all columns for unidentified characters e.g. "$"
- check all columns for Na-values e.g. NaN, NaT

Notes:
- doneChargingtime has NaT values --> possible interpretation: EV is discconnected from charging hub before reaching 100% state of charge
- what to do with IDs
- userID has NaN values --> not all users are registered

# Data Cleanup

## Step 1: check validity of columns

- remove "unnamed" column
- "sesionID" = combination of "id"
- check ranges of columns -->

In [5]:
# split data from "userInputs"

In [6]:
# only compile once
charging_set.drop('Unnamed: 0', axis=1, inplace=True)
charging_set.drop('sessionID', axis=1, inplace=True)
charging_set.head(3)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,timezone,userID,userInputs
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."


In [7]:
# ##### check ranges of columns
# print("id data range:", np.sort(charging_set["id"].unique()))
# print("connectionTime data range:", np.sort(charging_set["connectionTime"].unique()))
# print("disconnectTime data range:", np.sort(charging_set["disconnectTime"].unique()))
# print("doneChargingTime data range:", np.sort(charging_set["doneChargingTime"].unique()))
# print("kWhDelivered data range:", np.sort(charging_set["kWhDelivered"].unique()))
# print("sessionID data range:", np.sort(charging_set["sessionID"].unique()))
# print("siteID data range:", np.sort(charging_set["siteID"].unique()))
# print("spaceID data range:", np.sort(charging_set["spaceID"].unique()))
# print("stationID data range:", np.sort(charging_set["stationID"].unique()))
# print("timezone data range:", np.sort(charging_set["timezone"].unique()))
# print("userID data range:", np.sort(charging_set["userID"].unique()))
# print("userInputs data range:", np.sort(charging_set["userInputs"].unique()))
# print("connectionTime_date data range:", np.sort(charging_set["connectionTime_date"].unique()))
# print("connectionTime_time data range:", np.sort(charging_set["connectionTime_time"].unique()))
# print("disconnectTime_date data range:", np.sort(charging_set["disconnectTime_date"].unique()))
# print("disconnectTime_time data range:", np.sort(charging_set["disconnectTime_time"].unique()))
# print("doneChargingTime_date data range:", np.sort(charging_set["doneChargingTime_date"].unique()))
# print("doneChargingTime_time data range:", np.sort(charging_set["doneChargingTime_time"].unique()))

Since the "timezone" column only has the value "America/Los Angeles", we can remove this column.

In [8]:
# charging_set.drop('timezone', axis=1, inplace=True)
charging_set.head(3)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,timezone,userID,userInputs
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."


In [9]:
### check data types of each column
# Find the unique types
id_types = charging_set["id"].apply(type).unique()
connectionTime_types = charging_set["connectionTime"].apply(type).unique()
disconnectTime_types = charging_set["disconnectTime"].apply(type).unique()
doneChargingTime_types = charging_set["doneChargingTime"].apply(type).unique()
kWhDelivered_types = charging_set["kWhDelivered"].apply(type).unique()
siteID_types = charging_set["siteID"].apply(type).unique()
spaceID_types = charging_set["spaceID"].apply(type).unique()
stationID_types = charging_set["stationID"].apply(type).unique()
userID_types = charging_set["userID"].apply(type).unique()
userInputs_types = charging_set["userInputs"].apply(type).unique()

# Print the unique types
print("id types:", id_types)
print("connectionTime types:", connectionTime_types)
print("disconnectTime types:", disconnectTime_types)
print("doneChargingTime types:", doneChargingTime_types)
print("kWhDelivered types:", kWhDelivered_types)
print("siteID types:", siteID_types)
print("spaceID types:", spaceID_types)
print("stationID types:", stationID_types)
print("userID types:", userID_types)
print("userInputs types:", userInputs_types)

id types: [<class 'str'>]
connectionTime types: [<class 'str'>]
disconnectTime types: [<class 'str'>]
doneChargingTime types: [<class 'str'> <class 'float'>]
kWhDelivered types: [<class 'float'>]
siteID types: [<class 'int'>]
spaceID types: [<class 'str'>]
stationID types: [<class 'str'>]
userID types: [<class 'float'>]
userInputs types: [<class 'str'> <class 'float'>]


Next step: analyze "connectionTime", "disconnectTime" and "doneChargingTime", since these columns should hold time-datatypes and not strings nor floats (except for NaN-values since these have the type float).

# Time Data Preparation/Feature Engineering (draft)

For better data understanding and to improve data accessibility and interpretability, it is important to convert the current time data format into a human-readable time datatype. Currently the time related columns ("connectionTime", "disconnectionTime", "doneChargingTime") are formatted in the UTC format. To prepare the data for further interpretation we need to gather the relevant information. In this case the important information is the charging duration.

To get the duration we divide each of the time related columns into separate date and datetime columns. With these columns, the difference between connecting the EV with the station ("connectionTime") and disconnecting the EV with the station ("disconnectTime") can be calculated and stored in a separate column ("chargingDuration").


In [10]:
### split connectionTime in separate day and time columns
# Convert the column to datetime format
charging_set['connectionTime'] = pd.to_datetime(charging_set['connectionTime'])

# Create a new column for the date
charging_set['connectionTime_date'] = charging_set['connectionTime'].dt.date

# Create a new column for the time of the day
charging_set['connectionTime_time'] = charging_set['connectionTime'].dt.time


### split disconnectTime in separate day and time columns
# Convert the column to datetime format
charging_set['disconnectTime'] = pd.to_datetime(charging_set['disconnectTime'])

# Create a new column for the date
charging_set['disconnectTime_date'] = charging_set['disconnectTime'].dt.date

# Create a new column for the time of the day
charging_set['disconnectTime_time'] = charging_set['disconnectTime'].dt.time


### split doneChargingTime in separate day and time columns
# Convert the column to datetime format
charging_set['doneChargingTime'] = pd.to_datetime(charging_set['doneChargingTime'])

# Create a new column for the date
charging_set['doneChargingTime_date'] = charging_set['doneChargingTime'].dt.date

# Create a new column for the time of the day
charging_set['doneChargingTime_time'] = charging_set['doneChargingTime'].dt.time

In [11]:
### convert date columns into separate year, month and day columns

charging_set['connectionTime_year'] = charging_set['connectionTime'].dt.year
charging_set['connectionTime_month'] = charging_set['connectionTime'].dt.month
charging_set['connectionTime_day'] = charging_set['connectionTime'].dt.day

charging_set['disconnectTime_year'] = charging_set['disconnectTime'].dt.year
charging_set['disconnectTime_month'] = charging_set['disconnectTime'].dt.month
charging_set['disconnectTime_day'] = charging_set['disconnectTime'].dt.day

charging_set['doneChargingTime_year'] = charging_set['doneChargingTime'].dt.year
charging_set['doneChargingTime_month'] = charging_set['doneChargingTime'].dt.month
charging_set['doneChargingTime_day'] = charging_set['doneChargingTime'].dt.day

In [12]:
def calculate_hours():
    # Berechne die Ladezeit als Differenz zwischen disconnectTime und connectionTime
    charging_set["chargingDurationTemp"] = charging_set["disconnectTime"] - charging_set["connectionTime"]
    
    # Konvertiere die Differenz in Stunden
    charging_set['chargingDuration'] = charging_set["chargingDurationTemp"].apply(
        lambda x: x.total_seconds() / 3600
    )
    
    # Entferne die temporäre Spalte
    charging_set.drop(columns=["chargingDurationTemp"], inplace=True)
    
    return charging_set

# Aufruf der Funktion
charging_set = calculate_hours()

In [13]:
# clean "connectionTime", "disconnectTime", "doneChargingTime" from dataframe
charging_set = charging_set.drop(columns=["connectionTime", "disconnectTime", "doneChargingTime"])

In [14]:
### check column "doneChargingTime" for wrong datatype
# hypothesis: all float values in this column are nan-values
# Find the indices of float values in a column that should contain only strings
# float_indices = [idx for idx, value in charging_set['doneChargingTime'].items() if isinstance(value, float) and not isinstance(value, str)]

# Print the result
# if float_indices:
#     print("Indices of float values in column 'doneChargingTime' (all others should be strings):", float_indices)
# else:
#     print("No float values found in the column.")

In [15]:
# Get indices where 'column_name' has NaN values
# nan_indices = charging_set[charging_set['doneChargingTime'].isna()].index.tolist()

# Print the indices of NaN values
#  print(nan_indices)

In [16]:
# check for duplicate rows 

duplicate_counter = charging_set.duplicated().sum()
duplicates = charging_set[charging_set.duplicated()]
print("sum of duplicates:", duplicate_counter)
print(duplicates)

sum of duplicates: 1413
                             id  kWhDelivered  siteID  spaceID    stationID  \
65037  5d2fbdd3f9af8b4d0dd0d54f         2.463       1  AG-1F02  1-1-193-827   
65038  5d310f54f9af8b52adda3e20        12.891       1  AG-1F14  1-1-194-818   
65039  5d310f54f9af8b52adda3e21        13.734       1  AG-1F10  1-1-178-828   
65040  5d310f54f9af8b52adda3e22        10.619       1  AG-1F08  1-1-178-823   
65041  5d310f54f9af8b52adda3e23        19.079       1  AG-3F25  1-1-179-799   
...                         ...           ...     ...      ...          ...   
66445  5d574ad2f9af8b4c10c03652        28.787       1  AG-3F27  1-1-179-809   
66446  5d574ad2f9af8b4c10c03653         7.787       1  AG-3F30  1-1-179-810   
66447  5d574ad2f9af8b4c10c03654        11.274       1  AG-4F51  1-1-191-795   
66448  5d574ad2f9af8b4c10c03655        11.589       1  AG-4F43  1-1-191-778   
66449  5d574ad2f9af8b4c10c03656         0.897       1  AG-1F09  1-1-178-817   

                  timezone 

In [17]:
duplicates

,id,kWhDelivered,siteID,spaceID,stationID,timezone,userID,userInputs,connectionTime_date,connectionTime_time,...,connectionTime_year,connectionTime_month,connectionTime_day,disconnectTime_year,disconnectTime_month,disconnectTime_day,doneChargingTime_year,doneChargingTime_month,doneChargingTime_day,chargingDuration
65037,5d2fbdd3f9af8b4d0dd0d54f,2.463,1,AG-1F02,1-1-193-827,America/Los_Angeles,1117.0,"[{'WhPerMile': 200, 'kWhRequested': 4.0, 'mile...",2019-07-02,00:32:46,...,2019,7,2,2019,7,2,2019.0,7.0,2.0,2.036111
65038,5d310f54f9af8b52adda3e20,12.891,1,AG-1F14,1-1-194-818,America/Los_Angeles,2834.0,"[{'WhPerMile': 273, 'kWhRequested': 19.11, 'mi...",2019-07-02,12:15:28,...,2019,7,2,2019,7,2,2019.0,7.0,2.0,3.759167
65039,5d310f54f9af8b52adda3e21,13.734,1,AG-1F10,1-1-178-828,America/Los_Angeles,651.0,"[{'WhPerMile': 400, 'kWhRequested': 20.0, 'mil...",2019-07-02,12:42:49,...,2019,7,2,2019,7,2,2019.0,7.0,2.0,6.858333
65040,5d310f54f9af8b52adda3e22,10.619,1,AG-1F08,1-1-178-823,America/Los_Angeles,406.0,"[{'WhPerMile': 600, 'kWhRequested': 24.0, 'mil...",2019-07-02,12:44:46,...,2019,7,2,2019,7,2,2019.0,7.0,2.0,9.811944
65041,5d310f54f9af8b52adda3e23,19.079,1,AG-3F25,1-1-179-799,America/Los_Angeles,439.0,"[{'WhPerMile': 300, 'kWhRequested': 78.0, 'mil...",2019-07-02,12:52:53,...,2019,7,2,2019,7,2,2019.0,7.0,2.0,6.475556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66445,5d574ad2f9af8b4c10c03652,28.787,1,AG-3F27,1-1-179-809,America/Los_Angeles,393.0,"[{'WhPerMile': 240, 'kWhRequested': 31.2, 'mil...",2019-07-31,18:08:04,...,2019,7,31,2019,7,31,2019.0,7.0,31.0,5.353889
66446,5d574ad2f9af8b4c10c03653,7.787,1,AG-3F30,1-1-179-810,America/Los_Angeles,220.0,"[{'WhPerMile': 333, 'kWhRequested': 6.66, 'mil...",2019-07-31,18:40:41,...,2019,7,31,2019,8,1,2019.0,7.0,31.0,6.316944
66447,5d574ad2f9af8b4c10c03654,11.274,1,AG-4F51,1-1-191-795,America/Los_Angeles,1974.0,"[{'WhPerMile': 333, 'kWhRequested': 19.98, 'mi...",2019-07-31,19:04:40,...,2019,7,31,2019,7,31,2019.0,7.0,31.0,3.661667
66448,5d574ad2f9af8b4c10c03655,11.589,1,AG-4F43,1-1-191-778,America/Los_Angeles,942.0,"[{'WhPerMile': 275, 'kWhRequested': 22.0, 'mil...",2019-07-31,19:19:47,...,2019,7,31,2019,8,1,2019.0,7.0,31.0,5.251111


wir sollten uns erstmal anschauen, warum da so viele Duplikate sind , bevor wir sie droppen. 

In [18]:
charging_set.drop_duplicates()

,id,kWhDelivered,siteID,spaceID,stationID,timezone,userID,userInputs,connectionTime_date,connectionTime_time,...,connectionTime_year,connectionTime_month,connectionTime_day,disconnectTime_year,disconnectTime_month,disconnectTime_day,doneChargingTime_year,doneChargingTime_month,doneChargingTime_day,chargingDuration
0,5e23b149f9af8b5fe4b973cf,25.016,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil...",2020-01-02,13:08:54,...,2020,1,2,2020,1,2,2020.0,1.0,2.0,6.039167
1,5e23b149f9af8b5fe4b973d0,33.097,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil...",2020-01-02,13:36:50,...,2020,1,2,2020,1,2,2020.0,1.0,2.0,9.025278
2,5e23b149f9af8b5fe4b973d1,6.521,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",2020-01-02,13:56:35,...,2020,1,2,2020,1,3,2020.0,1.0,2.0,10.713056
3,5e23b149f9af8b5fe4b973d2,2.355,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",2020-01-02,13:59:58,...,2020,1,2,2020,1,2,2020.0,1.0,2.0,2.644722
4,5e23b149f9af8b5fe4b973d3,13.375,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil...",2020-01-02,14:00:01,...,2020,1,2,2020,1,2,2020.0,1.0,2.0,8.144167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65032,5d2fbdd3f9af8b4d0dd0d54a,8.399,1,AG-3F18,1-1-179-798,America/Los_Angeles,1346.0,"[{'WhPerMile': 400, 'kWhRequested': 32.0, 'mil...",2019-07-01,21:49:12,...,2019,7,1,2019,7,2,2019.0,7.0,1.0,3.801667
65033,5d2fbdd3f9af8b4d0dd0d54b,16.864,1,AG-3F20,1-1-179-794,America/Los_Angeles,364.0,"[{'WhPerMile': 400, 'kWhRequested': 40.0, 'mil...",2019-07-01,21:58:45,...,2019,7,1,2019,7,2,2019.0,7.0,2.0,2.684167
65034,5d2fbdd3f9af8b4d0dd0d54c,18.335,1,AG-4F47,1-1-191-807,America/Los_Angeles,2050.0,"[{'WhPerMile': 333, 'kWhRequested': 29.97, 'mi...",2019-07-01,22:02:21,...,2019,7,1,2019,7,2,2019.0,7.0,2.0,2.941389
65035,5d2fbdd3f9af8b4d0dd0d54d,22.815,1,AG-3F31,1-1-179-781,America/Los_Angeles,1626.0,"[{'WhPerMile': 200, 'kWhRequested': 34.0, 'mil...",2019-07-01,22:23:44,...,2019,7,1,2019,7,2,2019.0,7.0,2.0,3.664444


#### Anschauen User Inputs

In [19]:
charging_set["userInputs"].unique

<bound method Series.unique of 0        [{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil...
1        [{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil...
2        [{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...
3        [{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...
4        [{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil...
                               ...                        
66445    [{'WhPerMile': 240, 'kWhRequested': 31.2, 'mil...
66446    [{'WhPerMile': 333, 'kWhRequested': 6.66, 'mil...
66447    [{'WhPerMile': 333, 'kWhRequested': 19.98, 'mi...
66448    [{'WhPerMile': 275, 'kWhRequested': 22.0, 'mil...
66449                                                  NaN
Name: userInputs, Length: 66450, dtype: object>

In [20]:
import ast

In [21]:
def parse_user_inputs(row):
    try:
        # Parse the row as a Python object (e.g., list of dicts)
        parsed = ast.literal_eval(row) if pd.notnull(row) else []
        return parsed[0] if isinstance(parsed, list) and len(parsed) > 0 else {}
    except (ValueError, SyntaxError):
        return {}

# Parse 'userInputs' column in charging_set
charging_set['userInputs'] = charging_set['userInputs'].apply(parse_user_inputs)

# Specify the desired features to extract
desired_features = [
    'WhPerMile', 'kWhRequested', 'milesRequested', 
    'minutesAvailable', 'modifiedAt', 'paymentRequired', 
    'requestedDeparture'
]

# Extract desired features and create new columns
for feature in desired_features:
    charging_set[feature] = charging_set['userInputs'].apply(
        lambda x: x.get(feature) if isinstance(x, dict) else None
    )

In [22]:
# Drop the 'userInputs' column since it is no longer needed
charging_set = charging_set.drop(columns=['userInputs'])

In [23]:
charging_set.columns

Index(['id', 'kWhDelivered', 'siteID', 'spaceID', 'stationID', 'timezone',
       'userID', 'connectionTime_date', 'connectionTime_time',
       'disconnectTime_date', 'disconnectTime_time', 'doneChargingTime_date',
       'doneChargingTime_time', 'connectionTime_year', 'connectionTime_month',
       'connectionTime_day', 'disconnectTime_year', 'disconnectTime_month',
       'disconnectTime_day', 'doneChargingTime_year', 'doneChargingTime_month',
       'doneChargingTime_day', 'chargingDuration', 'WhPerMile', 'kWhRequested',
       'milesRequested', 'minutesAvailable', 'modifiedAt', 'paymentRequired',
       'requestedDeparture'],
      dtype='object')

In [24]:
charging_set

,id,kWhDelivered,siteID,spaceID,stationID,timezone,userID,connectionTime_date,connectionTime_time,disconnectTime_date,...,doneChargingTime_month,doneChargingTime_day,chargingDuration,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,5e23b149f9af8b5fe4b973cf,25.016,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,2020-01-02,13:08:54,2020-01-02,...,1.0,2.0,6.039167,250.0,25.00,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,5e23b149f9af8b5fe4b973d0,33.097,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,2020-01-02,13:36:50,2020-01-02,...,1.0,2.0,9.025278,280.0,70.00,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,5e23b149f9af8b5fe4b973d1,6.521,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,2020-01-02,13:56:35,2020-01-03,...,1.0,2.0,10.713056,400.0,8.00,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"
3,5e23b149f9af8b5fe4b973d2,2.355,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,2020-01-02,13:59:58,2020-01-02,...,1.0,2.0,2.644722,400.0,8.00,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"
4,5e23b149f9af8b5fe4b973d3,13.375,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,2020-01-02,14:00:01,2020-01-02,...,1.0,2.0,8.144167,400.0,16.00,40.0,504.0,"Thu, 02 Jan 2020 14:00:13 GMT",True,"Thu, 02 Jan 2020 22:24:01 GMT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66445,5d574ad2f9af8b4c10c03652,28.787,1,AG-3F27,1-1-179-809,America/Los_Angeles,393.0,2019-07-31,18:08:04,2019-07-31,...,7.0,31.0,5.353889,240.0,31.20,130.0,355.0,"Wed, 31 Jul 2019 18:08:23 GMT",True,"Thu, 01 Aug 2019 00:03:04 GMT"
66446,5d574ad2f9af8b4c10c03653,7.787,1,AG-3F30,1-1-179-810,America/Los_Angeles,220.0,2019-07-31,18:40:41,2019-08-01,...,7.0,31.0,6.316944,333.0,6.66,20.0,455.0,"Wed, 31 Jul 2019 18:41:02 GMT",True,"Thu, 01 Aug 2019 02:15:41 GMT"
66447,5d574ad2f9af8b4c10c03654,11.274,1,AG-4F51,1-1-191-795,America/Los_Angeles,1974.0,2019-07-31,19:04:40,2019-07-31,...,7.0,31.0,3.661667,333.0,19.98,60.0,184.0,"Wed, 31 Jul 2019 19:04:57 GMT",True,"Wed, 31 Jul 2019 22:08:40 GMT"
66448,5d574ad2f9af8b4c10c03655,11.589,1,AG-4F43,1-1-191-778,America/Los_Angeles,942.0,2019-07-31,19:19:47,2019-08-01,...,7.0,31.0,5.251111,275.0,22.00,80.0,60.0,"Wed, 31 Jul 2019 19:20:10 GMT",True,"Wed, 31 Jul 2019 20:19:47 GMT"


In [25]:
nan_user_id_indices = charging_set[charging_set['userID'].isna()].index[:10]

# Display the indices
print(nan_user_id_indices)

Index([11, 41, 180, 193, 265, 276, 407, 457, 474, 500], dtype='int64')


In [26]:
charging_set.loc[11]

id                        5e23b149f9af8b5fe4b973da
kWhDelivered                                 0.812
siteID                                           1
spaceID                                    AG-1F14
stationID                              1-1-194-818
timezone                       America/Los_Angeles
userID                                         NaN
connectionTime_date                     2020-01-02
connectionTime_time                       15:04:38
disconnectTime_date                     2020-01-02
disconnectTime_time                       23:33:26
doneChargingTime_date                   2020-01-02
doneChargingTime_time                     22:48:48
connectionTime_year                           2020
connectionTime_month                             1
connectionTime_day                               2
disconnectTime_year                           2020
disconnectTime_month                             1
disconnectTime_day                               2
doneChargingTime_year          

In [27]:
nan_counts = charging_set.isna().sum()
nan_counts_sorted = nan_counts.sort_values(ascending=False)

for column, count in nan_counts_sorted.items():
    print(f"Feature: {column}, Count of NaN-Values: {count}")

Feature: userID, Count of NaN-Values: 17263
Feature: kWhRequested, Count of NaN-Values: 17263
Feature: paymentRequired, Count of NaN-Values: 17263
Feature: modifiedAt, Count of NaN-Values: 17263
Feature: requestedDeparture, Count of NaN-Values: 17263
Feature: minutesAvailable, Count of NaN-Values: 17263
Feature: milesRequested, Count of NaN-Values: 17263
Feature: WhPerMile, Count of NaN-Values: 17263
Feature: doneChargingTime_month, Count of NaN-Values: 4088
Feature: doneChargingTime_date, Count of NaN-Values: 4088
Feature: doneChargingTime_time, Count of NaN-Values: 4088
Feature: doneChargingTime_day, Count of NaN-Values: 4088
Feature: doneChargingTime_year, Count of NaN-Values: 4088
Feature: siteID, Count of NaN-Values: 0
Feature: connectionTime_year, Count of NaN-Values: 0
Feature: disconnectTime_time, Count of NaN-Values: 0
Feature: connectionTime_time, Count of NaN-Values: 0
Feature: disconnectTime_date, Count of NaN-Values: 0
Feature: connectionTime_date, Count of NaN-Values: 0
F

#### Split for Users and Non-Users

- Binary ob User oder Non-User
- Alle User Input Daten werden mit -1 gefüllt, falls Nan

In [28]:
charging_set['UserID'] = charging_set['userID'].apply(lambda x: 1 if pd.notna(x) else 0)

In [29]:
desired_features = [
    'WhPerMile', 'kWhRequested', 'milesRequested', 
    'minutesAvailable', 'modifiedAt', 'paymentRequired', 
    'requestedDeparture'
]

# Convert each feature to 0 or 1 based on NaN values
for feature in desired_features:
    charging_set[f'{feature}_present'] = charging_set[feature].apply(lambda x: 1 if pd.notna(x) else 0)

In [30]:
nan_counts = charging_set.isna().sum()
nan_counts_sorted = nan_counts.sort_values(ascending=False)

for column, count in nan_counts_sorted.items():
    print(f"Feature: {column}, Count of NaN-Values: {count}")

Feature: milesRequested, Count of NaN-Values: 17263
Feature: minutesAvailable, Count of NaN-Values: 17263
Feature: paymentRequired, Count of NaN-Values: 17263
Feature: modifiedAt, Count of NaN-Values: 17263
Feature: requestedDeparture, Count of NaN-Values: 17263
Feature: userID, Count of NaN-Values: 17263
Feature: kWhRequested, Count of NaN-Values: 17263
Feature: WhPerMile, Count of NaN-Values: 17263
Feature: doneChargingTime_day, Count of NaN-Values: 4088
Feature: doneChargingTime_date, Count of NaN-Values: 4088
Feature: doneChargingTime_year, Count of NaN-Values: 4088
Feature: doneChargingTime_month, Count of NaN-Values: 4088
Feature: doneChargingTime_time, Count of NaN-Values: 4088
Feature: id, Count of NaN-Values: 0
Feature: timezone, Count of NaN-Values: 0
Feature: stationID, Count of NaN-Values: 0
Feature: spaceID, Count of NaN-Values: 0
Feature: siteID, Count of NaN-Values: 0
Feature: kWhDelivered, Count of NaN-Values: 0
Feature: connectionTime_year, Count of NaN-Values: 0
Featu

In [31]:
nan_chargingTime = charging_set[charging_set['doneChargingTime_time'].isna()].index[:10]

# Display the indices
print(nan_chargingTime)

Index([3756, 3757, 3758, 3759, 3760, 3761, 3762, 3763, 3764, 3765], dtype='int64')


In [32]:
charging_set.loc[3765]

id                                 5ea6278df9af8b4a2e7bd22c
kWhDelivered                                          4.725
siteID                                                    1
spaceID                                             AG-1F10
stationID                                       1-1-178-828
timezone                                America/Los_Angeles
userID                                                651.0
connectionTime_date                              2020-04-10
connectionTime_time                                14:55:07
disconnectTime_date                              2020-04-10
disconnectTime_time                                15:43:03
doneChargingTime_date                                   NaT
doneChargingTime_time                                   NaT
connectionTime_year                                    2020
connectionTime_month                                      4
connectionTime_day                                       10
disconnectTime_year                     

In [34]:
charging_set.to_csv('data/cleaned_charging_set.csv', index=False)